In [16]:
import os
import boto3
import pandas as pd
from botocore.exceptions import ClientError

access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if not aws_access_key_id or not aws_secret_access_key:
    print("AWS environment variables are not set.")
 
bucket_name = 'mindex-data-analytics-code-challenge'

s3_client = boto3.client('s3', 
    aws_access_key_id=access_key_id, 
    aws_secret_access_key=secret_access_key
)

try:
    s3_client.head_bucket(Bucket=bucket_name)
    print(f"The bucket '{bucket_name}' exists.")
except ClientError as exception:
    if exception.response['Error']['Code'] == '404':
        print(f"The bucket '{bucket_name}' does not exist.")
    else:
        print(f"An error occurred: {exception}")

file_keys = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']

dataframes = {}

tempfiles = {}

for key in file_keys:
    file_path = f'temp_{key.replace("/", "_")}'
    tempfiles[key] = file_path
    
    try:
        s3_client.download_file(bucket_name, key, file_path)
        print(f"File downloaded: {key}")
    except Exception as exception:
        print(f"An error occured downloading {key}: {exception}")
        
    try:
        df = pd.read_csv(file_path)
        dataframes[key] = df
    except Exception as exception:
        print(f"An error occured reading {file_path}")



while True:
    user_input = input("Delete temporary files?")

    if user_input.lower() == "yes" or "y":
        for key in file_keys:
            os.remove(tempfiles[key])
        print("Files delete, exiting program")
        break;
    else: 
        print("Exiting program")
        break;

        

The bucket 'mindex-data-analytics-code-challenge' exists.
File downloaded: bengals.csv
File downloaded: boyd_receiving.csv
File downloaded: chase_receiving.csv
File downloaded: higgins_receiving.csv


Delete temporary files? y


Files delete, exiting program
